In [1]:
naId = ""
parentNaId = ""
# define env vars
API_KEY_NA = "hzj5ASoq1aj5bHbl8XNI3roMecW9oAm7TOBMjkre"
# Define API endpoints
API_ENDPOINT_RECORD_NA = 'https://catalog.archives.gov/api/v2/records/search'
API_ENDPOINT_TRANSCRIPT_NA = 'https://catalog.archives.gov/api/v2/transcriptions/naId/'
API_ENDPOINT_COLLECTION_NA = 'https://catalog.archives.gov/api/v2/records/parentNaId/'

In [2]:
import requests
import polars as pl
import datetime as dt

def fetch_transcription_and_contributors(naId, api_key=API_KEY_NA, api_endpoint=API_ENDPOINT_TRANSCRIPT_NA):
    response = requests.get(api_endpoint + naId, headers={'Content-Type': 'application/json', 'x-api-key': api_key})
    if response.status_code == 200:
        data = response.json()
        if hit := data['body']['hits']['hits']:
            transcription_objects = []
            for x in hit:
                object_id = x['_source']['record']['target']['objectId']
                transcriptions_raw = x['_source']['record']['contribution']
                contributors_info = x['_source']['record']['contributors']
                transcription_object = {object_id: {"transcription":transcriptions_raw, "contributors_info": contributors_info}}
                transcription_objects.append(transcription_object)
            # its not clear how the transcription hits are ordered
            # we have to assume latest comes first
            return transcription_objects
        else:
            print("No hits found in response.")
            return None
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [3]:
parentNaId = "653144"

In [4]:
# df = pl.DataFrame(
#     {
#         "name": ["Alice Archer", "Ben Brown", "Chloe Cooper", "Daniel Donovan"],
#         "birthdate": [
#             dt.date(1997, 1, 10),
#             dt.date(1985, 2, 15),
#             dt.date(1983, 3, 22),
#             dt.date(1981, 4, 30),
#         ],
#         "weight": [57.9, 72.5, 53.6, 83.1],  # (kg)
#         "height": [1.56, 1.77, 1.65, 1.75],  # (m)
#     }
# )

# print(df)
# Fetch all pages of results and concatenate them into a single DataFrame
all_hits = []
page = 1
limit = 100  # Use a higher limit if the API allows

while True:
    params = {
        'page': page,
        'limit': limit,
    }
    response = requests.get(
        API_ENDPOINT_COLLECTION_NA + parentNaId,
        headers={'Content-Type': 'application/json', 'x-api-key': API_KEY_NA},
        params=params
    )
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
        break
    hits = response.json().get('body', {}).get('hits', {}).get('hits', [])
    if not hits:
        break
    all_hits.extend(hits)
    if len(hits) < limit:
        break  # Last page reached
    page += 1

if all_hits:
    try:
        df = pl.DataFrame(all_hits, strict=False)
    except Exception as e:
        print(f"Error creating DataFrame: {e}")
        df = pl.DataFrame()
else:
    print("No hits found in response.")
    df = pl.DataFrame()

In [5]:
len(df)

1234

In [6]:
df.head()

_index,_id,_score,_source,sort
str,str,f64,struct[2],list[str]
"""nac-records1""","""178210622""",10.162504,"{{""NAC_0052.xml"",{""records"",""recordGroup"",535},""2024-04-05 11:14:22.443596"",""2d18c2ee-1fff-4b02-8052-f8d5f5b42e09""},{""item"",""description"",{""Some or all of this material may be restricted by copyright or other intellectual property rights restrictions."",[""Copyright""],""Restricted - Possibly""},""True"",""Mr. L. Cramer"",null,[{null,[{""8601 Adelphi Road"",""College Park"",""301-837-3540"",""National Archives at College Park"",""20740-6001"",""National Archives at College Park - Motion Pictures"",""MD"",""301-837-3620"",""mopix@nara.gov"",""RDSM""}],[{1,[""Magnetic Media""],""Audio Tape/Reel""}],""Preservation""}],{""Unrestricted""},{null,""ou=NWCS-M,ou=groups"",""RDSM""},[""Sound Recordings""],""208.877A"",null,[{""208-877a.mp3"",""https://s3.amazonaws.com/NARAprodstorage/lz/mopix/208a/GENERALa/208-877a.mp3"",5828148,""Reel 1 of 1"",""244658952"",""Sound File (MP3)""}],[{{1926,""1926-01-01""},2,""recordGroup"",208,{1951,""1951-12-31""},""Records of the Office of War Information"",535,null}, {{1942,""1942-01-01""},1,""series"",null,{1946,""1946-12-31""},""Sound Recordings Relating to World War II"",653144,[{""Predecessor"",""Office for Emergency Management. Office of War Information. 6/13/1942-9/15/1945"",{9,1945,15,""1945-09-15""},""organization"",{6,1942,13,""1942-06-13""},10454749}, {""Most Recent"",""Department of State. Office of International Information and Cultural Affairs. (8/31/1945 - 8/15/1947)"",{8,1947,15,""1947-08-15""},""organization"",{8,1945,31,""1945-08-31""},10486776}]}],null,178210622,null,null,null,null}}","[""10.162504"", ""178210622""]"
"""nac-records1""","""178210618""",10.162504,"{{""NAC_ADHOC_2024-10-21_DESC_0029.xml"",{""records"",""recordGroup"",535},""2024-10-22 11:04:12.602766"",""df4ad273-d01f-4273-a07b-532c438297fe""},{""item"",""description"",{""Some or all of this material may be restricted by copyright or other intellectual property rights restrictions."",[""Copyright""],""Restricted - Possibly""},""True"",""West Indian Radio Newspaper"",""West Indian Radio Newspaper"",[{0,[{""8601 Adelphi Road"",""College Park"",""301-837-3540"",""National Archives at College Park"",""20740-6001"",""National Archives at College Park - Motion Pictures"",""MD"",""301-837-3620"",""mopix@nara.gov"",""RRSM""}],[{1,[""Magnetic Media""],""Audio Tape/Reel""}],""Preservation""}],{""Unrestricted""},{""National Archives at College Park – Motion Pictures"",""RRSM"",""RRSM""},[""Sound Recordings""],""208.875A"",[{11,1946,29,""1946-11-29""}],[{""208-875a.mp3"",""https://s3.amazonaws.com/NARAprodstorage/lz/mopix/208a/GENERALa/208-875a.mp3"",4128308,""Reel 1 of 1"",""244658944"",""Sound File (MP3)""}],[{{1926,""1926-01-01""},2,""recordGroup"",208,{1951,""1951-01-01""},""Records of the Office of War Information"",535,null}, {{1942,""1942-01-01""},1,""series"",null,{1946,""1946-01-01""},""Sound Recordings Relating to World War II"",653144,[{""Most Recent"",""Department of State. Office of International Information and Cultural Affairs. (08/31/1945 - 08/15/1947)"",{8,1947,15,""1947-08-15""},""organization"",{8,1945,31,""1945-08-31""},10486776}, {""Predecessor"",""Office for Emergency Management. Office of War Information. (06/13/1942 - 09/15/1945)"",{9,1945,15,""1945-09-15""},""organization"",{6,1942,13,""1942-06-13""},10454749}]}],""This item features Dr. Eric Williams."",178210618,null,null,null,null}}","[""10.162504"", ""178210618""]"
"""nac-records1""","""178210604""",10.162504,"{{""NAC_ADHOC_2024-10-21_DESC_0029.xml"",{""records"",""recordGroup"",535},""2024-10-22 11:04:12.584633"",""49f41ebb-84a4-4ca7-8d7a-4c6841450858""},{""item"",""description"",{""Some or all of this material may be restricted by copyright or other intellectual property rights restrictions."",[""Copyright""],""Restricted - Possibly""},""True"",""Joseph B. Eastman"",""America Speaks"",[{0,[{""8601 Adelphi Road"",""College Park"",

In [7]:
df = df.with_columns(
    pl.col('_source').map_elements(
        lambda x: x['record']['title'] if 'record' in x and 'title' in x['record'] else None
    ).alias('hit_title')
)

In [8]:
# apply this logic: hit_record_urls = [x['objectUrl'] for x in i['record']['digitalObjects']] to create a new column
df = df.with_columns(
    pl.col('_source').map_elements(
        lambda x: [obj['objectUrl'] for obj in (x['record'].get('digitalObjects') or [])] if 'record' in x and 'digitalObjects' in x['record'] else []
    ).alias('hit_record_urls')
)

In [9]:
df = df.with_columns(
    pl.col('_source').map_elements(
        lambda x: x['record'].get('digitalObjects') if 'record' in x else None
    ).alias('hit_digitalObjects_metadata')
)

In [ ]:
df = df.with_columns(
    pl.col('_id').map_elements(lambda naId: fetch_transcription_and_contributors(naId)).alias('transcription')
)

No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.
No hits found in response.


In [10]:
# save the dataframe as a parquet file
df.write_parquet(f'{parentNaId}_transcriptions.parquet', compression='snappy')

In [11]:
# retrieve the parquet file and read it back
df = pl.read_parquet(f'{parentNaId}_transcriptions.parquet')

In [12]:
df = df.explode("transcription")

ColumnNotFoundError: "transcription" not found

In [11]:
# filter out where transcription is None
df = df.filter(pl.col("transcription").is_not_null())

In [138]:
# filter out where transcription is any empty dict
df_transcriptions = df.filter(pl.col("transcription").map_elements(lambda x: bool(x) and isinstance(x, dict) and any(value is not None for value in x.values())))

/var/folders/w8/rhwrg1yd1vq5gr2f0q6b0ngw0000gn/T/ipykernel_35556/2082850090.py:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df_transcriptions = df.filter(pl.col("transcription").map_elements(lambda x: bool(x) and isinstance(x, dict) and any(value is not None for value in x.values())))


In [139]:
len(df_transcriptions)

2

In [ ]:
# for each item in the transcription column, extract the object_id and contributors_info

In [140]:
import os

def download_mp3(url):
    os.makedirs('./audio', exist_ok=True)
    filename = url.split('/')[-1]
    filepath = os.path.join('./audio', filename)
    response = requests.get(url)
    if response.status_code == 200:
        with open(filepath, 'wb') as f:
            f.write(response.content)
        return filepath
    else:
        print(f"Failed to download {url}: {response.status_code} - {response.text}")
        return None
    
# url_test = "https://s3.amazonaws.com/NARAprodstorage/lz/mopix/208a/GENERALa/208-192.mp3"

# # Download a test MP3 file
# downloaded_file = download_mp3(url_test)

# for each item in df_transcriptions['hit_record_urls'] (list of URLs), download the audio files and mark the filepaths in a new column 'audio_filepaths'
df_transcriptions = df_transcriptions.with_columns(
    pl.col('hit_record_urls').map_elements(
        lambda urls: [download_mp3(url) for url in urls if url]  # Filter out None URLs
    ).alias('audio_filepaths')
)

/var/folders/w8/rhwrg1yd1vq5gr2f0q6b0ngw0000gn/T/ipykernel_35556/1160828098.py:22: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df_transcriptions = df_transcriptions.with_columns(


In [141]:
# export df_transcriptions to a parquet file
df_transcriptions.write_parquet(f'{parentNaId}_transcriptions_with_audio.parquet', compression='snappy')

In [131]:
# df_transcriptions

In [ ]:


params = {
    'q': 'world war',
    # 'naID': ["2131159"],
    'rows': 10,
    'api_key': API_KEY_NA
}
# Make the API request
response = requests.get(API_ENDPOINT_RECORD_NA, params=params, headers={'Content-Type': 'application/json', 'x-api-key': API_KEY_NA})
# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    print("Records fetched successfully:")
    # print(response.json())
else:
    # Print the error message
    print(f"Error: {response.status_code} - {response.text}")
# This script fetches records from the National Archives and Records Administration (NARA) API

Records fetched successfully:


In [ ]:
# plan: for each collection, get all the records' naIds, fetch audio + metadata, and check if there are transcriptions
